In [9]:
# Prerequisite
%pip install google-genai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## The idea
1. Read the problem from problem.txt
2. Read the prompt from prompt.txt
3. Send the prompt as system instructions to the model
4. Receive the model's code and evaluate it
5. If the code is correct, print the result
6. If the code is not correct, send back the error message and ask the model to fix it
7. Repeat steps 4-6 until the code is correct or a maximum number of attempts is reached

In [10]:
# Import the required libraries
import os, sys, random
from google import genai
from google.genai import types

# Obtain API Key from .env file
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GENAI_API_KEY")

def generate(system_instruction, user_input, temp=1.25):
    client = genai.Client(
        api_key=api_key,
    )

    model = "gemini-2.5-flash-preview-05-20"

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=user_input),
            ],
        ),
    ]

    tools = [
        types.Tool(google_search=types.GoogleSearch()),
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=temp,
        tools=tools,
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text=system_instruction),
        ],
    )

    returns = []

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        # print(chunk.text, end="")
        if chunk.text:
            returns.append(chunk.text)
    
    return "".join(returns)

In [11]:
system_instruction = ""

with open("prompt-problem.txt", "r") as f:
    system_instruction = f.read()

with open("problem.txt", "r") as f:
    user_input = f.read()

expected_output = ""
output = ""
inputs = ""
error = ""

INPUT_FILE = "input.txt"
OUTPUT_FILE = "output.txt"
EXPECTED_FILE = "expected.txt"

def clean_code(code, lang="python"):
    # Loop through each line and remove all extra comments that is not within ````python` or `#`
    lines = code.splitlines()
    cleaned_lines = []
    foundCode = False
    for line in lines:
        if line.strip().startswith(f"```{lang}"):
            foundCode = True
        if not foundCode:
            continue
        cleaned_lines.append(line)
    code = "\n".join(cleaned_lines)
    # Check if the code starts with '```python' and ends with '```'
    if code.startswith(f'```{lang}'):
        code = code[len(f'```{lang}'):].strip()  # Remove the opening '```python' part
    if code.endswith('```'):
        code = code[:-3].strip()  # Remove the closing '```' part
    return code

# Evaluate the output
def evaluate_code(code):
    global inputs, output, expected_output, error

    original_stdin = sys.stdin
    original_stdout = sys.stdout
    
    cleaned_code = clean_code(code)

    print("--- Attempting to execute the following code: ---")
    print(cleaned_code)
    print("-------------------------------------------------")

    try:
        # Prepare input file for exec (and read its content for the retry prompt)
        if not os.path.exists(INPUT_FILE):
            print(f"Error: Input file '{INPUT_FILE}' not found.")
            error = f"Input file '{INPUT_FILE}' not found."
            # Set 'inputs' to a placeholder if it's used in error reporting or retry prompt
            inputs = f"Error: {INPUT_FILE} not found."
            return False
            
        with open(INPUT_FILE, "r") as f_in_for_prompt:
            inputs = f_in_for_prompt.read() # For the retry prompt
        
        # Redirect stdin for the exec call
        # Must open the file again for sys.stdin, as the previous 'with' block might have closed it or it was read to the end.
        sys.stdin = open(INPUT_FILE, "r")
        
        # Redirect stdout to capture the output of the exec call
        with open(OUTPUT_FILE, "w") as f_out:
            sys.stdout = f_out
            # Execute the cleaned code. Using an empty dict for globals for some isolation.
            exec(cleaned_code, {}) 
        
        # CRITICAL: Restore stdout and stdin immediately after exec and file operations
        sys.stdout = original_stdout
        sys.stdin = original_stdin # Close the file handle opened for stdin 

        with open(OUTPUT_FILE, "r") as f_out_read:
            output = f_out_read.read()
        
        if not os.path.exists(EXPECTED_FILE):
            print(f"Error: Expected output file '{EXPECTED_FILE}' not found.")
            error = f"Expected output file '{EXPECTED_FILE}' not found."
            return False

        with open(EXPECTED_FILE, "r") as f_expected:
            expected_output = f_expected.read()
        
        # Perform comparison
        if output.strip() == expected_output.strip():
            print("Output is correct.")
            return True
        else:
            print("Output does not match.")
            error = "Output mismatch"
            return False

    except Exception as e:
        # CRITICAL: Ensure restoration even if an error occurs
        sys.stdout = original_stdout
        # Also ensure stdin is restored (and its handle closed if we opened a new one)
        current_stdin_handle = sys.stdin
        sys.stdin = original_stdin
        if current_stdin_handle is not original_stdin and hasattr(current_stdin_handle, 'close'):
             # Check if it's not already closed (e.g., if error occurred after explicit close)
            if not current_stdin_handle.closed:
                current_stdin_handle.close()

        print(f"Error during code evaluation: {type(e).__name__}: {e}")
        error = f"{type(e).__name__}: {e}" # Capture error for the retry prompt
        
        # Attempt to read output.txt even if an error occurred, it might contain partial output or clues
        try:
            with open(OUTPUT_FILE, "r") as f_out_read_err:
                output = f_out_read_err.read()
        except Exception:
            output = "[Could not read output.txt after error, or it was empty/not created]"
            
        return False
    finally:
        # This ensures that sys.stdout and sys.stdin are *always* restored to their original states, even if an unexpected error (e.g. SystemExit, KeyboardInterrupt)
        # bypasses the `except Exception` block (though SystemExit and KeyboardInterrupt inherit from BaseException, not Exception).       
        # Store current sys.stdin to close it if it's a file we opened
        current_stdin_handle_finally = sys.stdin
        
        sys.stdout = original_stdout
        sys.stdin = original_stdin

        if current_stdin_handle_finally is not original_stdin and hasattr(current_stdin_handle_finally, 'close') and not current_stdin_handle_finally.closed:
            current_stdin_handle_finally.close()
    
solved = False
retries = 0
best_code = None

while not solved:
    # Generate code
    if retries > 0:
        improved_prompt = f"Inputs: {inputs}, Outputs: {output}, Expected: {expected_output}, Error: {error}, Code: {best_code}, Problem: {user_input}"
    else:
        improved_prompt = user_input

    code = generate(system_instruction, improved_prompt, temp=random.uniform(0.5, 1.5))
    
    # Evaluate the code
    solved = evaluate_code(code)
    
    if not solved:
        print(f"Inputs: {inputs}")
        print(f"Outputs: {output}")
        print(f"Expected: {expected_output}")
        print(f"Error: {error}")
        print("Retrying...")
        retries += 1
        best_code = code
        if retries > 10:
            print("Max retries reached. Exiting.")
            break
        continue
    else:
        print("Code is correct.")
        best_code = code
        break

--- Attempting to execute the following code: ---
import sys

def solve():
    # Read n and k from stdin
    n, k = map(int, sys.stdin.readline().split())
    
    # Read the array a from stdin
    a = list(map(int, sys.stdin.readline().split()))

    # Find the minimum and maximum number of apples in any box
    min_val = min(a)
    max_val = max(a)

    # If the initial difference between max and min is greater than k+1,
    # the current player (Tom) cannot make any valid move without immediately losing.
    # Therefore, Jerry wins.
    if max_val - min_val > k + 1:
        sys.stdout.write("Jerry\n")
    else:
        # If max_val - min_val is less than or equal to k+1,
        # it has been proven that there is always at least one safe move for the current player.
        # This means the game will always continue until all apples are collected.
        # The winner is then determined by the total number of apples.
        # If the total sum is odd, Tom (the first player) takes th

## Further Improvements
1. Add a remove comment function to remove comments from the code [/]
2. Convert the code to C++ for better execution speed

In [12]:
# Remove comments
best_code = clean_code(code)

def remove_comments_and_docstrings(code_lines):
    result = []
    inside_docstring = False
    
    for line in code_lines:
        # Skip empty lines
        if not line.strip():
            continue
        
        # Check if we are inside a docstring
        if '"""' in line:
            inside_docstring = not inside_docstring
            continue  # Skip the docstring line
        
        # Skip lines that are comments or inside docstrings
        if inside_docstring or line.strip().startswith("#"):
            continue
        
        # Add non-comment and non-docstring lines to the result
        result.append(line)
    
    return result

cleaned_code_lines = remove_comments_and_docstrings(best_code.split("\n"))
cleaned_code = "\n".join(cleaned_code_lines)
print(cleaned_code)

import sys
def solve():
    n, k = map(int, sys.stdin.readline().split())
    a = list(map(int, sys.stdin.readline().split()))
    min_val = min(a)
    max_val = max(a)
    if max_val - min_val > k + 1:
        sys.stdout.write("Jerry\n")
    else:
        total_apples = sum(a)
        if total_apples % 2 == 1:
            sys.stdout.write("Tom\n")
        else:
            sys.stdout.write("Jerry\n")
num_test_cases = int(sys.stdin.readline())
for _ in range(num_test_cases):
    solve()


In [13]:
with open("prompt-convert.txt", "r") as f:
    prompt_converter = f.read()

if solved:
    converted_code = generate(prompt_converter, cleaned_code)
    print(clean_code(converted_code, lang="cpp"))

#include <bits/stdc++.h>

using namespace std;

void solve() {
    int n, k;
    cin >> n >> k;
    vector<int> a(n);
    long long total_apples = 0;
    int min_val = INT_MAX;
    int max_val = INT_MIN;

    for (int i = 0; i < n; ++i) {
        cin >> a[i];
        total_apples += a[i];
        if (a[i] < min_val) {
            min_val = a[i];
        }
        if (a[i] > max_val) {
            max_val = a[i];
        }
    }

    if (max_val - min_val > k + 1) {
        cout << "Jerry\n";
    } else {
        if (total_apples % 2 == 1) {
            cout << "Tom\n";
        } else {
            cout << "Jerry\n";
        }
    }
}

int main() {
    ios_base::sync_with_stdio(false);
    cin.tie(NULL);

    int num_test_cases;
    cin >> num_test_cases;
    while (num_test_cases--) {
        solve();
    }

    return 0;
}
